# 元々のjsonファイルを取得し，整形したものを出力

・ラベル名をtagsに統一

・ラベルとして使用されていた数値IDを文字型に変換

In [ ]:
#{"words": ["CHICAGO", "AT", "ATLANTA"], "ner": ["B-ORG", "O", "B-LOC"]}

In [5]:
import json

In [19]:
with open("data/original/id2coarse_tags.json","r") as f:
    label_coarse =json.loads(f.read())
    
with open("data/original/id2fine_tags.json","r") as f:
    label_fine =json.loads(f.read())

In [22]:
def coarse_str_tag(tags):
    new_tag = [label_coarse[str(i)] for i in tags]
    return new_tag
    
def fine_str_tag(tags):
    new_tag = [label_fine[str(i)] for i in tags]
    return new_tag

In [23]:
for filename in ["train","test","dev"]:
    
    data = []
    with open("data/original/"+filename+".json","r") as f:
        for line in f:
            data.append(json.loads(line))

    coarse = []
    for d in data:
        new_d = {}
        new_d["tokens"] = d["tokens"]
        new_d["tags"] = coarse_str_tag(d["coarse_tags"])
        coarse.append(new_d)
    with open("data/coarse/"+filename+".json","w") as f:
        for d in coarse:
            f.write(json.dumps(d))
            f.write("\n")

    fine = []
    for d in data:
        new_d = {}
        new_d["tokens"] = d["tokens"]
        new_d["tags"] = fine_str_tag(d["fine_tags"])
        fine.append(new_d)
    with open("data/fine/"+filename+".json","w") as f:
        for d in fine:
            f.write(json.dumps(d))
            f.write("\n")

In [7]:
label_coarse

[{'0': 'O',
  '1': 'art',
  '2': 'building',
  '3': 'event',
  '4': 'location',
  '5': 'organization',
  '6': 'other',
  '7': 'person',
  '8': 'product'}]

# Simple-BI

・BIラベルのみに着目したデータ拡張

・交換率100％

In [1]:
import json
import random

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def Simple_BI(dataname,filename,n):
    data = []
    with open("data/"+dataname+"/"+filename+".json","r") as f:
        for line in f:
            data.append(json.loads(line))
            
    alterna_labels = {}
    if dataname == "c":
        tag_json = "data/original/id2coarse_tags.json"
    else:
        tag_json = "data/original/id2fine_tags.json"
    with open(tag_json,"r") as f:
        label_coarse =json.loads(f.read())
        label_list = list(label_coarse.values())[1:]
    for label in label_list:
        alterna_labels[label] = set()
    for d in data:
        for token,tag in zip(d["tokens"],d["tags"]):
            if tag in label_list:
                alterna_labels[tag].add(token)

    data_f_path = "data/"+dataname+"_BI_x"+str(n)+"/"+filename+".json"
    memo_f_path = "log/"+dataname+"_BI_x"+str(n)+"_"+filename+"_memo.txt"
    
    
    with open(data_f_path,"w") as data_f, open(memo_f_path,"w") as memo_f:
        for d in data:
            data_f.write(json.dumps(d))
            data_f.write("\n")

        add_n = n*len(data)-len(data)
        print()
        print(dataname,filename,"add_n",add_n)
        
        for now in range(add_n):
            if now%10 == 0:
                print('\r%d / %d' %(now, add_n), end='')

            x = random.randint(0,len(data)-1)
            d = data[x]
            new_d = dict()
            new_d["tokens"] = []
            new_d["tags"] = data[x]["tags"]

            cnt = 0

            for token,tag in zip(d["tokens"],d["tags"]):
                if tag != "O":
                    if tag in alterna_labels:
                        token=random_choice_token(alterna_labels[tag]-{token})
                        cnt = cnt + 1
                    new_d["tokens"].append(token)

            data_f.write(json.dumps(new_d))
            data_f.write("\n")

            memo_f.write(str(x)+"\t"+" ".join(d["tokens"])+"\n")
            memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d["tokens"])+"\n")
            memo_f.write("\n")

        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [6]:
# for dataname in ["f"]:
#     for filename in ["dev","train"]:
#         Simple_BI(dataname,filename,2)
#         Simple_BI(dataname,filename,5)
#         Simple_BI(dataname,filename,10)

# TreePos-BI

・f直下のデータを取得し，各文を表すdict型を一つずつ取り出しd["id"]という文書番号を表すキーと値を追加

In [25]:
import json

data = []
filename = "dev"
# filename = "train"
with open("data/f/"+filename+".json","r") as f:
    for line in f:
        data.append(json.loads(line))
        
for i,d in enumerate(data):
    d["id"] = i

・idのキーを持つdict型の要素を500ずつまとめたjsonとし，data/f_idに保存

In [26]:
n = 500
for i in range(0, len(data), n):
    with open("data/f_id/"+filename+"_"+str(i)+"-"+str(i+n-1)+".json","w") as f:
        for d in data[i: i+n]:
                f.write(json.dumps(d))
                f.write("\n")

・f_id内の500ずつデータが入った細切れのjsonを全て統合してtrain.json,dev,jsonへ

In [15]:
import os

path = "data/f_id/
filenames = os.listdir(path)

dev_data_list = []
train_data_list = []

for filename in filenames:
    print(filename)
    with open(path + filename,'rb') as f:
        data = pickle.load(f)
    if "dev" in filename:   
        dev_data_list = dev_data_list + data
    else:
        train_data_list = train_data_list + data
        
with open(path + "dev.json", 'w') as ef:
    for d in dev_data_list:
        ef.write(json.dumps(d))
        ef.write("\n")
        
with open(path + "train.json", 'w') as ef:
    for d in train_data_list:
        ef.write(json.dumps(d))
        ef.write("\n")

・ 形態素情報付与

In [8]:
from nltk.parse.stanford import *
import nltk
from nltk.tree import *
import pickle
import os

In [9]:
java_path = "jre1.8.0_333/bin/java.exe"
os.environ['JAVAHOME'] = java_path

parser =  'stanford-corenlp/stanford-parser-full-2020-11-17/stanford-parser.jar'
parser_model = 'stanford-corenlp/stanford-corenlp-4.2.0-models-english.jar'

#POSタグの分析用
pos = StanfordParser(path_to_jar=parser, path_to_models_jar = parser_model)
#係り受け関係の分析用
dep_parser = StanfordDependencyParser(path_to_jar=parser, path_to_models_jar = parser_model)

def POSTagAnalysis(text):
    out = pos.raw_parse(text)
    out = list(out)
    tree = out[0]
    return tree

/tmp/ipykernel_7991/2297462148.py:8: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  pos = StanfordParser(path_to_jar=parser, path_to_models_jar = parser_model)
/tmp/ipykernel_7991/2297462148.py:10: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  dep_parser = StanfordDependencyParser(path_to_jar=parser, path_to_models_jar = parser_model)


・filenameを受け取り，構文解析情報を付与し，Stanford_coreNLP内部に構文情報とともに保存

In [39]:
import json


def TreePOS(filename):
    data = []
    with open("data/f_id/"+filename,"r") as f:
        for line in f:
            data.append(json.loads(line))
            
    new_document=[]
    document_trees = []
    print(filename)
    
    #タプル型で複数引数を渡す
    for i,s in enumerate(data):
        print('\r%d / %d' %(i, len(data)), end='')
        
        text = " ".join(s["tokens"])
        tree = POSTagAnalysis(text)
        document_trees.append(tree)
        new_d = dict()

        new_d["tokens"] = []
        new_d["tree_pos"] = []
        new_d["tags"] = []

        for i in range(len(tree.pos())):
            new_d["tokens"].append(tree.pos()[i][0])
            new_d["tree_pos"].append(tree.pos()[i][1])

        new_d["tags"]=s["tags"]
        new_d["id"] = s["id"]
        new_document.append(new_d)
        del tree
        
        
    with open("data/Stanford_coreNLP/"+filename+"_tree.binaryfile", 'wb') as f:
        pickle.dump(document_trees,f)
    with open("data/Stanford_coreNLP_TreePos/"+filename+"_data.binaryfile", 'wb') as f:
        pickle.dump(new_document,f)

    print("\nSAVED ==> "+filename+"_tree.binaryfile")
    print("SAVED ==> "+filename+"_data.binaryfile")

・errorがなくなるまで繰り返し構文解析

In [ ]:
import os
filenames = os.listdir("data/f_id/")
errors = []
all_errors = []

In [71]:
for filename in filenames:
    try:
        TreePOS(filename)
    except:
        errors.append(filename)
        
all_errors.append(errors)
filenames = errors
errors = []

In [70]:
 with open("data/TreePosError.binaryfile", 'wb') as f:
        pickle.dump(errors,f)

・ Stanford_coreNLP内を全て統合して{train|dev}.json，{train_tree|dev_tree}.binaryfileへ

In [105]:
import os
tree_path = "data/Stanford_coreNLP/"

dev_tree_list = []
train_tree_list = []

filenames = os.listdir(tree_path)
for filename in filenames:
    with open(tree_path + filename,'rb') as f:
        data = pickle.load(f)
        if "dev" in filename:   
            dev_tree_list = dev_tree_list + data
        else:
            train_tree_list = train_tree_list + data

In [1]:
import os
import pickle
data_path = "data/Stanford_coreNLP_TreePos/"

dev_data_list = []
train_data_list = []

filenames = os.listdir(data_path)
for filename in filenames:
    with open(data_path + filename,'rb') as f:
        data = pickle.load(f)
        if "dev" in filename:   
            dev_data_list = dev_data_list + data
        else:
            train_data_list = train_data_list + data

In [3]:
import json
with open(data_path + "dev.json", 'w') as ef:
    for d in dev_data_list:
        ef.write(json.dumps(d))
        ef.write("\n")
with open(data_path + "train.json", 'w') as ef:
    for d in train_data_list:
        ef.write(json.dumps(d))
        ef.write("\n")

In [107]:
with open(tree_path+"dev_tree.binaryfile", 'wb') as f:
    pickle.dump(dev_tree_list,f)
    
with open(tree_path+"train_tree.binaryfile", 'wb') as f:
    pickle.dump(train_tree_list,f)

・ f_id内を全て統合してtrain.json,dev,jsonへ

・ 上の位置にコピーを動かし済み．実行後削除

In [143]:
import os

path = "data/f_id/"
filenames = os.listdir(path)

dev_origin_list = []
train_origin_list = []

for filename in filenames:
    data = []
    with open("data/f_id/"+filename,"r") as f:
        for line in f:
            data.append(json.loads(line))
        if "dev" in filename:
            dev_origin_list = dev_origin_list + data
        else:
            train_origin_list = train_origin_list + data
        
with open(path + "dev.json", 'w') as ef:
    for d in dev_origin_list:
        ef.write(json.dumps(d))
        ef.write("\n")
        
with open(path + "train.json", 'w') as ef:
    for d in train_origin_list:
        ef.write(json.dumps(d))
        ef.write("\n")

・ tokenが構文解析器入力前と入力後で異なるものを取得

In [106]:
dev_origin_list = []
with open("data/f_id/dev.json","r") as f:
    for line in f:
        dev_origin_list.append(json.loads(line))
        
train_origin_list = []
with open("data/f_id/train.json","r") as f:
    for line in f:
        train_origin_list.append(json.loads(line))

In [112]:
print(len(dev_data_list))
print(len(train_data_list))

18823
131766


In [146]:
print(len(dev_origin_list))
print(len(train_origin_list))

18823
131766


・dev_bebigger.json 　　dev_besmaller.json 　　dev_same.json を作成

In [439]:
bebigger = []
besmaller = []
same=[] #id tokens tags
for d in dev_origin_list:
    #構文解析によってtoken数が変化する問題に対処する必要がある
    #解析前tokenと解析後tokenを比較し，等しくない場合は
    #解析前後の大小関係ごとにリストに格納し，結果の比較用データとする
    
    new_data = data_d[d["id"]]
    d["new_tokens"] = new_data["tokens"]
    if len(d['tokens']) < len(d["new_tokens"]):
        bebigger.append(d)
    elif len(d['tokens']) > len(d["new_tokens"]):
        besmaller.append(data_d[d["id"]])
    else:
        same.append(data_d[d["id"]])

In [440]:
# bebigger:3833
with open("data/Stanford_coreNLP_interm/dev_bebigger.json", 'w') as ef:
    for d in bebigger:
        ef.write(json.dumps(d))
        ef.write("\n")
# besmaller:1
with open("data/Stanford_coreNLP_interm/dev_besmaller.json", 'w') as ef:
    for d in besmaller:
        ef.write(json.dumps(d))
        ef.write("\n")
#same:14989
with open("data/Stanford_coreNLP_interm/dev_same.json", 'w') as ef:
    for d in same:
        ef.write(json.dumps(d))
        ef.write("\n")

・train_bebigger.json 　　train_besmaller.json 　　train_same.json を作成

In [465]:
data_d = dict()
for d in train_data_list:
    data_d[d["id"]] = d

bebigger = []
besmaller = []
same=[] #id tokens tags
for d in train_origin_list:
    #構文解析によってtoken数が変化する問題に対処する必要がある
    #解析前tokenと解析後tokenを比較し，等しくない場合は
    #解析前後の大小関係ごとにリストに格納し，結果の比較用データとする
    
    new_data = data_d[d["id"]]
    d["new_tokens"] = new_data["tokens"]
    d["new_tags"] = new_data["tags"]
    d["tree_pos"] = new_data["tree_pos"]
    if len(d['tokens']) < len(d["new_tokens"]):
        bebigger.append(d)
    elif len(d['tokens']) > len(d["new_tokens"]):
        besmaller.append(d)
    else:
        same.append(d)

In [467]:
len(same)

104918

In [468]:
# bebigger:26841
with open("data/Stanford_coreNLP_interm/train_bebigger.json", 'w') as ef:
    for d in bebigger:
        ef.write(json.dumps(d))
        ef.write("\n")
# besmaller:7
with open("data/Stanford_coreNLP_interm/train_besmaller.json", 'w') as ef:
    for d in besmaller:
        ef.write(json.dumps(d))
        ef.write("\n")
#same:104918
with open("data/Stanford_coreNLP_interm/train_same.json", 'w') as ef:
    for d in same:
        ef.write(json.dumps(d))
        ef.write("\n")

・{dev|train}_bebiggerについて，新たなトークン数のラベルに合わせる

・{dev|train}_bebiggerにnew_tagsを付与．うまくいったもの，そうでないもので{dev|train}_successと{dev|train}_errorに分割

In [469]:
#単純に細分化問題のみに対応して新たなタグを付与し，うまくいったものとそうでないものを返す関数
def GiveNewTag(input_list):
    success = []
    error = []
    for d in input_list:
        new_tags = [""]*len(d["new_tokens"])

        old_i = 0
        new_i = 0
        try:
            for _ in d["new_tokens"]:
                if d["new_tokens"][new_i]==d["tokens"][old_i]:
                    new_tags[new_i] = d["tags"][old_i]
                    old_i += 1
                    new_i += 1
                else:
                    if d["new_tokens"][new_i] in d["tokens"][old_i]:
                        new_tags[new_i] = d["tags"][old_i]
                        new_i += 1
                    else:
                        old_i += 1
                        if d["new_tokens"][new_i]==d["tokens"][old_i]:
                            new_tags[new_i] = d["tags"][old_i]
                            old_i += 1
                            new_i += 1


            if len(new_tags)==len(d["new_tokens"]):
                d["new_tags"] = new_tags
                success.append(d)
            else:
                error.append(d)
        except:
            error.append(d)
    return [success,error]

In [470]:
#dev=>success/error
import json
        
dev_bebigger = []
with open("data/Stanford_coreNLP_interm/dev_bebigger.json","r") as f:
    for line in f:
        dev_bebigger.append(json.loads(line))

dev_success,dev_error = GiveNewTag(dev_bebigger)#3001,#820

In [471]:
#train=>success/error
import json

train_bebigger = []
with open("data/Stanford_coreNLP_interm/train_bebigger.json","r") as f:
    for line in f:
        train_bebigger.append(json.loads(line))
        
train_success,train_error = GiveNewTag(train_bebigger)#21006,#5835

In [472]:
len(train_error)

5835

・{dev|train}_errorのうち括弧が"-LRB-","-RRB-"になる問題を解決　うまくいったものを{dev|train}_PBに，そうでないもの{dev|train}_error_PBへ

In [473]:
#細分化問題+タグ変更問題に対応，新たなタグを付与し，うまくいったものとそうでないものを返す関数
def GiveNewTag_PB(input_list):
    success = []
    error = []
    for d in input_list:
        new_tags = [""]*len(d["new_tokens"])

        old_i = 0
        new_i = 0
        try:
            for _ in d["new_tokens"]:
                if d["new_tokens"][new_i]==d["tokens"][old_i] or d["new_tokens"][new_i] in ("-LRB-","-RRB-"):
                    new_tags[new_i] = d["tags"][old_i]
                    old_i += 1
                    new_i += 1
                else:
                    if d["new_tokens"][new_i] in d["tokens"][old_i]:
                        new_tags[new_i] = d["tags"][old_i]
                        new_i += 1
                    else:
                        old_i += 1
                        if d["new_tokens"][new_i]==d["tokens"][old_i]:
                            new_tags[new_i] = d["tags"][old_i]
                            old_i += 1
                            new_i += 1


            if len(new_tags)==len(d["new_tokens"]):
                d["new_tags"] = new_tags
                success.append(d)
            else:
                error.append(d)
        except:
            error.append(d)
    return [success,error]

In [474]:
#dev_error=>PB/error_PB
dev_PB,dev_error_PB= GiveNewTag_PB(dev_error)#820,#12

In [475]:
#train_error=>PB/error_PB
train_PB,train_error_PB = GiveNewTag_PB(train_error)#820,#12

・dev_error_PBを修正

・Dod. . のようにカンマが被る問題を削除

In [476]:
#重複カンマ削除
def Remove_comme(input_list):
    for d in input_list:
        remake_tokens = []
        for i in range(len(d["new_tokens"])):
            token = d["new_tokens"][i]
            if token != ".":
                remake_tokens.append(token)
        if d["tokens"][-1] == ".":
            remake_tokens.append(".")
        d["new_tokens"] = remake_tokens
    return input_list

dev_error_PB= Remove_comme(dev_error_PB)
train_error_PB= Remove_comme(train_error_PB)

・カンマを削除したnew_tokenに対し改めてnew_tagsを付与

In [477]:
dev_comma,dev_error_comma = GiveNewTag_PB(dev_error_PB)
train_comma,train_error_comma = GiveNewTag_PB(train_error_PB)

In [478]:
#devを保存
with open("data/Stanford_coreNLP_interm/dev_bebigger_only_3001.json", 'w') as ef:
    for d in dev_success:
        ef.write(json.dumps(d))
        ef.write("\n")
        
with open("data/Stanford_coreNLP_interm/dev_BP_only_820.json", 'w') as ef:
    for d in dev_PB:
        ef.write(json.dumps(d))
        ef.write("\n")
        
with open("data/Stanford_coreNLP_interm/dev_comma_9.json", 'w') as ef:
    for d in dev_comma:
        ef.write(json.dumps(d))
        ef.write("\n")
                
with open("data/Stanford_coreNLP_interm/dev_fraction_3.json", 'w') as ef:
    for d in dev_error_comma:
        ef.write(json.dumps(d))
        ef.write("\n")

#trainを保存
with open("data/Stanford_coreNLP_interm/train_bebigger_only_26841.json", 'w') as ef:
    for d in train_success:
        ef.write(json.dumps(d))
        ef.write("\n")
        
with open("data/Stanford_coreNLP_interm/train_BP_only_5750.json", 'w') as ef:
    for d in train_PB:
        ef.write(json.dumps(d))
        ef.write("\n")
        
with open("data/Stanford_coreNLP_interm/train_comma_61.json", 'w') as ef:
    for d in train_comma:
        ef.write(json.dumps(d))
        ef.write("\n")
                
with open("data/Stanford_coreNLP_interm/train_fraction_24.json", 'w') as ef:
    for d in train_error_comma:
        ef.write(json.dumps(d))
        ef.write("\n")

・{train|dev}_sameにエラーが無いかを調査

In [479]:
dev_same = []
with open("data/Stanford_coreNLP_interm/dev_same.json","r") as df:
    for line in df:
        dev_same.append(json.loads(line))

train_same = []
with open("data/Stanford_coreNLP_interm/train_same.json", 'r') as tf:
    for line in tf:
        train_same.append(json.loads(line))

In [7]:
dev_same_unmatch = []
dev_same_match = []
for d in dev_same:
    if d["tokens"] != d["new_tokens"]:
        dev_same_unmatch.append(d)
    else:
        dev_same_match.append(d)

In [147]:
train_same_unmatch = []
train_same_match = []
for d in train_same:
    if d["tokens"] != d["new_tokens"]:
        train_same_unmatch.append(d)
    else:
        train_same_match.append(d)

・dev_same_match,train_same_matchを保存

In [201]:
with open("data/Stanford_coreNLP_interm/dev_same_match_13024.json", 'w') as ef:
    for d in dev_same_match:
        ef.write(json.dumps(d))
        ef.write("\n")
        
with open("data/Stanford_coreNLP_interm/train_same_match_91892.json", 'w') as ef:
    for d in train_same_match:
        ef.write(json.dumps(d))
        ef.write("\n")

・dev_same_unmatch,train_same_unmatchのうち，記号のtoken変化とそうでないものに分類

In [193]:
dev_unmatch_PB,dev_unmatch_error_PB = GiveNewTag_PB(dev_same_unmatch)
train_unmatch_PB,train_unmatch_error_PB = GiveNewTag_PB(train_same_unmatch)

・記号変化対処で解決できたものを保存

In [430]:
with open("data/Stanford_coreNLP_interm/dev_unmatch_PB_1918.json", 'w') as ef:
    for d in dev_unmatch_PB:
        ef.write(json.dumps(d))
        ef.write("\n")
        
with open("data/Stanford_coreNLP_interm/train_unmatch_PB_13022.json", 'w') as ef:
    for d in train_unmatch_PB:
        ef.write(json.dumps(d))
        ef.write("\n")

・対応できなかったもの

In [204]:
with open("data/Stanford_coreNLP_interm/dev_unmatch_error_PB_2.json", 'w') as ef:
    for d in dev_unmatch_error_PB:
        ef.write(json.dumps(d))
        ef.write("\n")
        
with open("data/Stanford_coreNLP_interm/train_unmatch_error_PB_2.json", 'w') as ef:
    for d in train_unmatch_error_PB:
        ef.write(json.dumps(d))
        ef.write("\n")

・以上，これまでに出てきた処理しきれないデータについて目視で確認

In [316]:
dev_besmaller = []
with open("data/Stanford_coreNLP_interm/dev_besmaller.json","r") as f:
    for line in f:
        dev_besmaller.append(json.loads(line))

train_besmaller = []
with open("data/Stanford_coreNLP_interm/train_besmaller.json","r") as f:
    for line in f:
        train_besmaller.append(json.loads(line))
        
dev_fraction_3 = []
with open("data/Stanford_coreNLP_interm/dev_fraction_3.json","r") as f:
    for line in f:
        dev_fraction_3.append(json.loads(line))

train_fraction_24 = []
with open("data/Stanford_coreNLP_interm/train_fraction_24.json","r") as f:
    for line in f:
        train_fraction_24.append(json.loads(line))
        
dev_unmatch_error_PB = []
with open("data/Stanford_coreNLP_interm/dev_unmatch_error_PB_2.json","r") as f:
    for line in f:
        dev_unmatch_error_PB.append(json.loads(line))

train_unmatch_error_PB = []
with open("data/Stanford_coreNLP_interm/train_unmatch_error_PB_2.json","r") as f:
    for line in f:
        train_unmatch_error_PB.append(json.loads(line))

In [317]:
# # dev_unmatch_error_PB => OK
# for i in range(2):
#     print(len(dev_unmatch_error_PB[i]["tokens"]),"_".join(dev_unmatch_error_PB[i]["tokens"]))
#     print(len(dev_unmatch_error_PB[i]["tokens"]),"_".join(dev_unmatch_error_PB[i]["new_tokens"]))
#     print()
    
# # train_unmatch_error_PB = OK
# for i in range(2):
#     print(len(train_unmatch_error_PB[i]["tokens"]),"_".join(train_unmatch_error_PB[i]["tokens"]))
#     print(len(train_unmatch_error_PB[i]["tokens"]),"_".join(train_unmatch_error_PB[i]["new_tokens"]))
#     print()

In [318]:
# new_tagsを目視で入力するための出力関数
def Add_new_tags(d):
    print("ID:",d["id"])
    new_tags = [""]*len(d["new_tokens"])
    
    olg_token_and_tag = ""
    for i in range(len(d["tokens"])):
        olg_token_and_tag += " "+d["tokens"][i]+"_ "+d["tags"][i]+"  "
    print('\r%s' %(olg_token_and_tag), end='')
    
    for j in range(len(d["new_tokens"])):
        t = input(d["new_tokens"][j]+" = ")
        if t == "":
            new_tags[j] = "O"
        else:
             new_tags[j] = t
    d["new_tags"] = new_tags 
    return d

・devにおいて目視チェックが必要なものを全て確認し，dev_checked_1+3+2.jsonとして保存

In [323]:
# dev_checked = []

# for d in dev_fraction_3:
#     d = Add_new_tags(d)
#     dev_checked.append(d)

# for d in dev_besmaller:
#     d = Add_new_tags(d)
#     dev_checked.append(d)

# for d in dev_unmatch_error_PB:
#     dev_checked.append(d)

with open("data/Stanford_coreNLP_interm/dev_checked_1+3+2.json", 'w') as ef:
    for d in dev_checked:
        ef.write(json.dumps(d))
        ef.write("\n")

・trainにおいて目視チェックが必要なものを全て確認し，train_checked_7+24+2.jsonとして保存

In [349]:
# train_checked = []#24
# for d in train_fraction_24:
#     d = Add_new_tags(d)
#     train_checked.append(d)

In [368]:
# other_checked = []
# target_checked = []
# for d in train_checked:
#     if d["id"] in (44882,58227,94353,94720,99241,101971,119067):
#         target_checked.append(d)
#     else:
#         other_checked.append(d)

In [373]:
# target_checked_remake = []

In [395]:
# i = 6 #(0～7)
# d = target_checked[i]#44882,58227,94353,94720,99241,101971,119067
# print(i)
# print(d["id"])
# new_d = Add_new_tags(d)
# target_checked_remake.append(new_d)

In [410]:
# train_besmaller_checked = []
# for d in train_besmaller:
#     new_d = Add_new_tags(d)
#     train_besmaller_checked.append(new_d)

In [ ]:
train_unmatch_error_PB = []
with open("data/Stanford_coreNLP_interm/train_unmatch_error_PB_2.json","r") as f:
    for line in f:
        train_unmatch_error_PB.append(json.loads(line))

In [413]:
new_train_checked = other_checked + target_checked_remake + train_besmaller_checked + train_unmatch_error_PB

with open("data/Stanford_coreNLP_interm/train_checked_7+24+2.json", 'w') as ef:
    for d in new_train_checked:
        ef.write(json.dumps(d))
        ef.write("\n")

・{train|dev}ともに処理済みのデータを全て結合し，保存

In [5]:
dev = []#18823

with open("data/Stanford_coreNLP_interm/dev_bebigger_only_3001.json","r") as f:
    for line in f:
        dev.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/dev_BP_only_820.json","r") as f:
    for line in f:
        dev.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/dev_comma_9.json","r") as f:
    for line in f:
        dev.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/dev_checked_1+3+2.json","r") as f:
    for line in f:
        dev.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/dev_same_match_13024.json","r") as f:
    for line in f:
        dev.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/dev_unmatch_PB_1918.json","r") as f:
    for line in f:
        dev.append(json.loads(line))
        
with open("data/Stanford_coreNLP_interm/dev.json", 'w') as ef:
    for d in dev:
        ef.write(json.dumps(d))
        ef.write("\n")

In [434]:
train = []#131766

with open("data/Stanford_coreNLP_interm/train_bebigger_only_26841.json","r") as f:
    for line in f:
        train.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/train_BP_only_5750.json","r") as f:
    for line in f:
        train.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/train_comma_61.json","r") as f:
    for line in f:
        train.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/train_checked_7+24+2.json","r") as f:
    for line in f:
        train.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/train_same_match_91892.json","r") as f:
    for line in f:
        train.append(json.loads(line))

with open("data/Stanford_coreNLP_interm/train_unmatch_PB_13022.json","r") as f:
    for line in f:
        train.append(json.loads(line))
        
with open("data/Stanford_coreNLP_interm/train.json", 'w') as ef:
    for d in train:
        ef.write(json.dumps(d))
        ef.write("\n")

## TreePos_Tagの作成

・（ここから再開する場合は）token整形，統合結果であるdevとtrainの読み込み

In [90]:
dev = []
with open("data/Stanford_coreNLP_interm/dev.json","r") as f:
    for line in f:
        dev.append(json.loads(line))
        
train = []
with open("data/Stanford_coreNLP_interm/train.json","r") as f:
    for line in f:
        train.append(json.loads(line))

・構文解析結果であるdev_dataとtrain_dataの読込み・辞書化

In [83]:
dev_data = []
with open("data/Stanford_coreNLP_TreePos/dev.json","r") as f:
    for line in f:
        dev_data.append(json.loads(line))
        
train_data = []
with open("data/Stanford_coreNLP_TreePos/train.json","r") as f:
    for line in f:
        train_data.append(json.loads(line))
        
dev_d = dict()
for d in dev_data:
    dev_d[d["id"]] = d
    
train_d = dict()
for d in train_data:
    train_d[d["id"]] = d

・整形後にtree_posを追加

In [91]:
for d in dev:
    d["tree_pos"]=dev_d[d["id"]]["tree_pos"]
    if "new_tokens" in d:
        d["tokens"] = d["new_tokens"]
        del d["new_tokens"]
    if "new_tags" in d:
        d["tags"] = d["new_tags"]
        del d["new_tags"]

for d in train:
    d["tree_pos"]=train_d[d["id"]]["tree_pos"]
    if "new_tokens" in d:
        d["tokens"] = d["new_tokens"]
        del d["new_tokens"]
    if "new_tags" in d:
        d["tags"] = d["new_tags"]
        del d["new_tags"]

・#Stanford_coreNLP内に作成された構文解析済みデータから，新たにTreePos_Tagを持つデータを作成
#全て統合してtrain.json,dev.jsonへ

・tokensとtree_posが合わないものはerrorへ

In [93]:
#Stanford_coreNLP内に作成された構文解析済みデータから，新たにTreePos_Tagを持つデータを作成
#全て統合してtrain.json,dev,jsonへ
def ModelFormat(data):
    new_data = []
    error_data = []
    for d in data:
        new_d = dict()
        new_d["tokens"] = d["tokens"]
        if len(d["tokens"]) == len(d["tree_pos"]):
            new_d["tags"] = [str(d["tree_pos"][j])+"_"+str(d["tags"][j]) for j in range(len(d["tags"]))]
            new_data.append(new_d)
        else:
            error_data.append(new_d)
    return [new_data,error_data]

In [94]:
new_dev,error_dev = ModelFormat(dev)
with open("data/TreePos_BI/dev.json", 'w', encoding="utf-8") as wf:
    for d in new_dev:
        wf.write(json.dumps(d))
        wf.write("\n")
        
with open("data/TreePos_BI/dev_error.json", 'w',encoding="utf-8") as ef:
    for d in error_dev:
        ef.write(json.dumps(d))
        ef.write("\n")
    
new_train,error_train = ModelFormat(train)
with open("data/TreePos_BI/train.json", 'w',encoding="utf-8") as wf:
    for d in new_train:
        wf.write(json.dumps(d))
        wf.write("\n")
        
with open("data/TreePos_BI/train_error.json", 'w',encoding="utf-8") as ef:
    for d in error_train:
        ef.write(json.dumps(d))
        ef.write("\n")

・データ拡張実装前準備

・alterna_labelsの作成

In [95]:
dev_data = []
with open("data/TreePos_BI/dev.json","r") as f:
    for line in f:
        dev_data.append(json.loads(line))
train_data = []
with open("data/TreePos_BI/train.json","r") as f:
    for line in f:
        train_data.append(json.loads(line))
        
label_list = []
for d in dev_data:
    label_list.extend(d["tags"])
for d in train_data:
    label_list.extend(d["tags"])
label_list = list(set([tags for tags in label_list if "_O" not in tags]))

alterna_labels = {}
for label in label_list:
    alterna_labels[label] = set()
for d in dev_data+train_data:
    for token,tag in zip(d["tokens"],d["tags"]):
        if tag in label_list:
            alterna_labels[tag].add(token)

・ここで，alterna_labelsのうち，候補が2つ以下のものは削除（下の例の2・3行目）

{'VBZ_person-actor': {"'s",  'Brest',  'Mature',  'Torrens',  'cameos',  'follows',  'heartthrobs',  'is',  'stars'},

'RBR_art-film': {'Better', 'Worse'},

'TO_person-athlete': {'to'}

...}

In [96]:
new_alterna_labels = dict()
for k,v in alterna_labels.items():
    if len(v)>2:
        new_alterna_labels[k] = v

alterna_labels = new_alterna_labels

・データ拡張実装

In [106]:
import json
import random

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def TreePos_BI(filename,n):
    data = []
    with open("data/TreePos_BI/"+filename+".json","r",encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))

    data_f_path = "data/TreePos_BI_x"+str(n)+"/"+filename+".json"
    memo_f_path = "log/TreePos_BI_x"+str(n)+"_"+filename+"_memo.txt"
    
    
    with open(data_f_path,"w",encoding="utf-8") as data_f, open(memo_f_path,"w",encoding="utf-8") as memo_f:
        for d in data:
            data_f.write(json.dumps(d))
            data_f.write("\n")

        add_n = n*len(data)-len(data)
        print()
        print(filename,"add_n",add_n)
        
        for now in range(add_n):
            if now%10 == 0:
                print('\r%d / %d' %(now, add_n), end='')

            x = random.randint(0,len(data)-1)
            d = data[x]
            new_d = dict()
            new_d["tokens"] = []
            new_d["tags"] = data[x]["tags"]
            cnt = 0

            for token,tag in zip(d["tokens"],d["tags"]):
                if tag in alterna_labels:
                    token=random_choice_token(alterna_labels[tag]-{token})
                    cnt = cnt + 1
                new_d["tokens"].append(token)

            data_f.write(json.dumps(new_d))
            data_f.write("\n")

            memo_f.write(str(x)+"\t"+" ".join(d["tokens"])+"\n")
            memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d["tokens"])+"\n")
            memo_f.write("\n")

        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [107]:
TreePos_BI("dev",2)


dev add_n 18813
18810 / 18813

In [108]:
TreePos_BI("train",2)


train add_n 131705
131700 / 131705

In [ ]:
#ラベルの長さ違うエラー出たので確認

In [1]:
import json

origin = []
with open("data/f/dev.json","r") as f:
    for line in f:
        origin.append(json.loads(line))
        
dev = []
with open("data/Stanford_coreNLP/dev.json","r") as f:
    for line in f:
        dev.append(json.loads(line))
        
error = []
with open("data/Stanford_coreNLP/error_dev.json","r") as f:
    for line in f:
        error.append(json.loads(line))

In [7]:
print(len(origin))
print(len(dev))
print(len(error))

18823
29338
7485


In [9]:
error_num = []
error_data = []
for i,d in enumerate(data):
    
    if len(d["tokens"])!=len(d["tags"]):
        error_num.append(i)
        error_data.append(d)

In [6]:
len(error_num)

7484

In [7]:
len(data)

73644

In [10]:
# error_data

In [19]:
len(error_data[4]["tokens"])

81

In [20]:
len(error_data[4]["tags"])

75

In [29]:
import pickle
error_data = []
for path_sub in ["dev"]:
    path = "data/Stanford_coreNLP/"

    data = []
    with open("data/f/dev.json","r") as f:
        for line in f:
            data.append(json.loads(line))
        
    new_data = []
    
    for d in data:
        new_d = dict()
        new_d["tokens"] = d["tokens"]
        if len(d["tokens"])==len(d["tags"]):
            new_d["tags"] = d["tags"]
            new_data.append(new_d)
            
        else:
            error_data.append(d)
            
    with open(path + "dev_2.json","w") as wf:
        for d in new_data:
            wf.write(json.dumps(d))
            wf.write("\n")
            
    with open(path + "error_dev_2.json", 'w') as ef:
        for d in error_data:
            ef.write(json.dumps(d))
            ef.write("\n")